In [1]:
from transformers import Trainer
from datasets import load_dataset
import torch

ds = load_dataset("gokaygokay/prompt-enhancer-dataset") 

2024-12-13 22:16:18.043541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734149778.170137    3909 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734149778.213129    3909 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 22:16:18.325181: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [3]:
def tokenize_function(examples):
    # Tokenize both the 'short_prompt' and 'long_prompt' columns
    #short_prompt_tokens = tokenizer(examples["short_prompt"], padding="max_length", truncation=True)
    long_prompt_tokens = tokenizer(examples["long_prompt"], padding="max_length", truncation=True)
    
    # Merge both tokenized outputs into a single dictionary (or add them separately)
    return {
        #'short_prompt_input_ids': short_prompt_tokens['input_ids'],
        #'short_prompt_attention_mask': short_prompt_tokens['attention_mask'],
        'long_prompt_input_ids': long_prompt_tokens['input_ids'],
        'long_prompt_attention_mask': long_prompt_tokens['attention_mask']
    }

# Tokenize the entire dataset (train and test)
tokenized_datasets = ds.map(tokenize_function, batched=True)


In [4]:
def preprocess_data(examples):
    # The model takes input_ids and attention_mask as inputs, and long_prompt_input_ids as labels
    model_inputs = {
        'input_ids': examples['long_prompt_input_ids'],
        'attention_mask': examples['long_prompt_attention_mask'],
        'labels': examples['long_prompt_input_ids']  # The long_prompt_input_ids are the target labels
    }
    return model_inputs

# Apply the preprocessing to the train and test datasets
tokenized_datasets = tokenized_datasets.map(preprocess_data, batched=True)

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['short_prompt', 'long_prompt', 'long_prompt_input_ids', 'long_prompt_attention_mask', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16110
    })
    test: Dataset({
        features: ['short_prompt', 'long_prompt', 'long_prompt_input_ids', 'long_prompt_attention_mask', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1790
    })
})

In [6]:
from datasets import DatasetDict, Dataset

# Your dataset (replace with actual data)
dataset = DatasetDict({
    'train': Dataset.from_dict({
        'short_prompt_input_ids': tokenized_datasets['train']['long_prompt_input_ids'], 
        'short_prompt_attention_mask': tokenized_datasets['train']['long_prompt_attention_mask'], 
        'long_prompt_input_ids': tokenized_datasets['train']['long_prompt_input_ids']
    }),
    'test': Dataset.from_dict({
        'short_prompt_input_ids': tokenized_datasets['test']['long_prompt_input_ids'], 
        'short_prompt_attention_mask': tokenized_datasets['test']['long_prompt_attention_mask'], 
        'long_prompt_input_ids': tokenized_datasets['test']['long_prompt_input_ids']
    })
})


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids'],
        num_rows: 16110
    })
    test: Dataset({
        features: ['short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids'],
        num_rows: 1790
    })
})

In [8]:
def preprocess_data(examples):
    return {
        'input_ids': examples['short_prompt_input_ids'],
        'attention_mask': examples['short_prompt_attention_mask'],
        'labels': examples['long_prompt_input_ids']
    }

# Apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/16110 [00:00<?, ? examples/s]

Map:   0%|          | 0/1790 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets 

DatasetDict({
    train: Dataset({
        features: ['short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16110
    })
    test: Dataset({
        features: ['short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1790
    })
})

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt_results",              # Save the model here
    evaluation_strategy="epoch",         # Evaluate after each epoch
    learning_rate=2e-5,                  # Learning rate
    per_device_train_batch_size=8,       # Training batch size
    per_device_eval_batch_size=8,        # Evaluation batch size
    num_train_epochs=3,                  # Number of epochs
    weight_decay=0.001,                   # Regularization
    logging_dir="./t5_logs",                # Logs directory
    logging_steps=5000,                   # Log every 500 steps
)


/home/dsu/Project/PeacockFlock/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
) 


In [12]:
trainer.train()

  0%|          | 0/6042 [00:00<?, ?it/s]

  0%|          | 0/224 [00:00<?, ?it/s]

{'eval_loss': 0.3246106803417206, 'eval_runtime': 42.5325, 'eval_samples_per_second': 42.086, 'eval_steps_per_second': 5.267, 'epoch': 1.0}


  0%|          | 0/224 [00:00<?, ?it/s]

{'eval_loss': 0.3174147605895996, 'eval_runtime': 42.2097, 'eval_samples_per_second': 42.407, 'eval_steps_per_second': 5.307, 'epoch': 2.0}
{'loss': 0.3407, 'grad_norm': 0.5809955596923828, 'learning_rate': 3.4491890102615033e-06, 'epoch': 2.48}


  0%|          | 0/224 [00:00<?, ?it/s]

{'eval_loss': 0.31529486179351807, 'eval_runtime': 42.5667, 'eval_samples_per_second': 42.052, 'eval_steps_per_second': 5.262, 'epoch': 3.0}
{'train_runtime': 3912.5361, 'train_samples_per_second': 12.353, 'train_steps_per_second': 1.544, 'train_loss': 0.33704355259440427, 'epoch': 3.0}


TrainOutput(global_step=6042, training_loss=0.33704355259440427, metrics={'train_runtime': 3912.5361, 'train_samples_per_second': 12.353, 'train_steps_per_second': 1.544, 'total_flos': 2.525648781312e+16, 'train_loss': 0.33704355259440427, 'epoch': 3.0})

In [13]:
trainer.evaluate() 

  0%|          | 0/224 [00:00<?, ?it/s]

{'eval_loss': 0.31529486179351807,
 'eval_runtime': 42.4412,
 'eval_samples_per_second': 42.176,
 'eval_steps_per_second': 5.278,
 'epoch': 3.0}

In [14]:
model.save_pretrained("./fine_tuned_gpt")
tokenizer.save_pretrained("./fine_tuned_gpt")


('./fine_tuned_gpt/tokenizer_config.json',
 './fine_tuned_gpt/special_tokens_map.json',
 './fine_tuned_gpt/vocab.json',
 './fine_tuned_gpt/merges.txt',
 './fine_tuned_gpt/added_tokens.json',
 './fine_tuned_gpt/tokenizer.json')

: 